In [ ]:
import itertools 
import random
import math 


class Vertex:
    """
    Class representing a vertex in a graph.
    """
    def __init__(self, id):
        """
        Constructor for the Vertex class.
        Args:
            id (str): Unique identifier for the vertex.
        """
        self.id = id
        self.neighbors = []  # List of vertex neighbors
        self.grade = len(self.neighbors)


    def add_neighbor(self, neighbor):
        """
        Adds a neighbor to the vertex's neighbor list.
        
        Args:
            neighbor (Vertex): Neiprint(graph)ghbor vertex object.
        """
        self.neighbors.append(neighbor)

    def __str__(self):
        neighbors = ", ".join([n.id for n in self.neighbors])
        return f'Vertex({self.id}) -> [{neighbors}]'



class Edge:
    """
    Class representing an edge in a graph.
    """
    def __init__(self, source, target, id, directed=False):
        """
        Constructor for the Edge class.
        Args:
            source (Vertex): Source vertex.
            target (Vertex): Target vertex.
            id (str): Unique identifier for the edge.
            directed (bool): Defines whether the edge is directed.
        """
        self.source = source
        self.target = target
        self.id = id
        self.directed = directed

    def __str__(self):
        direction = "<->" if self.directed else "--"
        return f'{self.source.id}{direction}{self.target.id}'







 

class Graph:
    """
    Class representing a graph with vertices and edges.
    """
    def __init__(self, directed=False):
        """
        Constructor for the Graph class.
        Args:
            directed (bool): Defines whether the graph is directed.
        """
        self.vertices = {}  # Dictionary of vertices
        self.edges = {}  # Dictionary of edges
        self.directed = directed

    def add_vertex(self, id):
        """
        Adds a vertex to the graph.
        
        Args:
            id (str): Vertex identifier.
        """
        if id not in self.vertices:
            self.vertices[id] = Vertex(id)

    def add_edge(self, source_id, target_id):
        """
        Adds an edge between two vertices.
        
        Args:
            source_id (str): ID of the source vertex.
            target_id (str): ID of the target vertex.
        """
        if source_id not in self.vertices:
            self.add_vertex(source_id)
        if target_id not in self.vertices:
            self.add_vertex(target_id)

        source = self.vertices[source_id]
        target = self.vertices[target_id]

        edge_id = f'{source_id}<->{target_id}' if self.directed else f'{source_id}--{target_id}'
        self.edges[edge_id] = Edge(source, target, edge_id, self.directed)
        
        source.add_neighbor(target)
        if not self.directed:
            target.add_neighbor(source)

        
    def gradeVertex(self, id):
        vertex = self.vertices.get(id)

        if not vertex:
            return 0
    
        else:
            return len(vertex.neighbors)

    def __str__(self):
        graph_type = 'Directed' if self.directed else 'Undirected'
        vertices = ", ".join(self.vertices.keys())
        edges = ", ".join(str(edge) for edge in self.edges.values())
        return f'Graph ({graph_type}):\n  Vertices = [{vertices}]\n  Edges = [{edges}]'
    

    def generate_dot(self, filename='graph.dot'):
        """
        Generates a DOT format file for the graph.
        
        Args:
            filename (str): Output file name.
        """
        dot_graph = "digraph G {\n" if self.directed else "graph G {\n"
        
        for vertex in self.vertices.values():
            dot_graph += f'    "{vertex.id}";\n'

        for edge in self.edges.values():
            connector = " -> " if edge.directed else " -- "
            dot_graph += f'    "{edge.source.id}"{connector}"{edge.target.id}";\n'
        
        dot_graph += "}"
        
        with open(filename, 'w') as f:  # Corrección aquí
            f.write(dot_graph)
        
        print(f'DOT file saved as {filename}')
        

    def meshGraph(self, numRows, numColumns):
            """
            Creates a mesh graph with specified rows and columns.
            
            Args:
                rows (int): Number of rows in the mesh.
                columns (int): Number of columns in the mesh.
            """
            for row in range(numRows):
                for column in range(numColumns - 1):
                    source = f'{row}_{column}'
                    target_horizontal = f'{row}_{column + 1}'
                    self.add_edge(source, target_horizontal)

            for column in range(numColumns):
                for row in range(numRows - 1):
                    source = f'{row}_{column}'
                    target_vertical = f'{row + 1}_{column}'
                    self.add_edge(source, target_vertical)



    def ErdosRenyiGraph(self, numVertices, numEdges):
        """
        Generates an Erdős–Rényi random graph G(n, m) as a method of the Graph class.

        Args:
            n (int): Number of vertices.
            m (int): Number of edges.
        """
        # Vertex creation        
        for vertice in range(numVertices):
            self.add_vertex(str(vertice))

        # 
        ids = list(self.vertices.keys())
        possibleEdges = list(itertools.combinations(ids, 2)) 

        selectedEdges = random.sample(possibleEdges, numEdges)


        for source_id, target_id in selectedEdges:
            self.add_edge(source_id, target_id)



    def GilbertGraph(self, numVertices, prob ):
        """_summary_

        Args:
            numVertices (int): Number of vertices
            prob (float): Probability associated to create an edge
        """

        # Vertex Creation
        for i in range(numVertices):
            self.add_vertex(str(i))
        ids = list(self.vertices.keys())

        # Create all the possible edge pairs
        posiblesEdges = itertools.combinations(ids, 2)

        # Para cada par, lanzar una moneda con probabilidad `p`
        for source_id, target_id in posiblesEdges:
            if random.random() <= prob:
                self.add_edge(source_id, target_id)



    def geographicGraph(self, n, r):
        """
        Generates a random graph using the simple geographic model G(n, r).
        
        Args:
            n (int): Number of nodes (> 0).
            r (float): Maximum distance to create an edge (0 < r <= 1).
        """



        # Assign random positions (x, y) in [0,1] x [0,1]
        positions = {}
        for i in range(n):
            node_id = str(i)
            self.add_vertex(node_id)
            positions[node_id] = (random.random(), random.random())

        ids = list(positions.keys())

        # Check each pair and create an edge if distance ≤ r
        for i in range(n):
            for j in range(i + 1, n) if not self.directed else range(n):
                if not self.directed and i >= j:
                    continue
                source_id = ids[i]
                target_id = ids[j]
                x1, y1 = positions[source_id]
                x2, y2 = positions[target_id]
                distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

                if distance <= r:
                    self.add_edge(source_id, target_id)

    def BarabasiAlbertGraph(self, numVertices, maxGrade):
        # Iterate through the number of vertices to be added
        for vertex in range(numVertices):
            # Add the new vertex to the graph
            self.add_vertex(str(vertex))

            # Try to connect the new vertex to existing ones
            for existingVertexKey in self.vertices.keys():
                # Avoid self-loops and check max degree constraint
                if existingVertexKey != str(vertex) and self.gradeVertex(existingVertexKey) < maxGrade:
                    # Compute probability based on inverse of current degree
                    probsCreateGraph = 1 - (self.gradeVertex(existingVertexKey)) / maxGrade
                    randomProb = random.random()

                    # Add edge with the computed probability
                    if randomProb < probsCreateGraph:
                        self.add_edge(str(vertex), existingVertexKey)



    def DorogovtsevMendesGraph(self, numVertex):
        # Require at least 3 vertices to start the base triangle
        if numVertex < 3:
            raise ValueError("Number of vertices must be at least 3.")
        
        # Create the initial triangle
        self.add_vertex("0")    
        self.add_vertex("1")
        self.add_vertex("2")

        self.add_edge("0", "1")
        self.add_edge("1", "2")
        self.add_edge("2", "0")
        
        # Add new vertices one by one
        for i in range(3, numVertex):
            # Choose a random existing vertex
            randomVertex1Key = random.choice(list(self.vertices.keys()))
            randomVertex1 = self.vertices[randomVertex1Key]
            
            # Choose one of its neighbors
            randomVertex2 = random.choice(list(randomVertex1.neighbors))
            
            # Add the new vertex
            self.add_vertex(str(i))
            
            # Connect it to both endpoints of the chosen edge
            self.add_edge(str(i), randomVertex1.id)
            self.add_edge(str(i), randomVertex2.id)

### DorogovtsevMendesGraph

In [196]:
grafoTriangulos50=Graph()
grafoTriangulos50.DorogovtsevMendesGraph(50)
grafoTriangulos50.generate_dot('DorogovtsevMendesGraph50.dot')


grafoTriangulos200=Graph()
grafoTriangulos200.DorogovtsevMendesGraph(200)
grafoTriangulos200.generate_dot('DorogovtsevMendesGraph200.dot')


grafoTriangulos500=Graph()
grafoTriangulos500.DorogovtsevMendesGraph(500)
grafoTriangulos500.generate_dot('DorogovtsevMendesGraph500.dot')

DOT file saved as DorogovtsevMendesGraph50.dot
DOT file saved as DorogovtsevMendesGraph200.dot
DOT file saved as DorogovtsevMendesGraph500.dot


### Malla

In [44]:
# Grafo de malla
graphMesh50 = Graph(directed=False)
graphMesh50.meshGraph(5,10)
graphMesh50.generate_dot('Mesh50.dot')

graphMesh200 = Graph(directed=False)
graphMesh200.meshGraph(10,20)
graphMesh200.generate_dot('Mesh200.dot')


graphMesh500 = Graph(directed=False)
graphMesh500.meshGraph(25,20)
graphMesh500.generate_dot('Mesh500.dot')

DOT file saved as Mesh50.dot
DOT file saved as Mesh200.dot
DOT file saved as Mesh500.dot


### Erdos

In [60]:
# Grafo de ErdosRenyi
ErdosRenyi50 = Graph(directed=False)
ErdosRenyi50.ErdosRenyiGraph(50, 200)
ErdosRenyi50.generate_dot('ErdosRenyi50.dot')


ErdosRenyi200 = Graph(directed=False)
ErdosRenyi200.ErdosRenyiGraph(300, 1200)
ErdosRenyi200.generate_dot('ErdosRenyi200.dot')


ErdosRenyi500 = Graph(directed=False)
ErdosRenyi500.ErdosRenyiGraph(500, 1500)
ErdosRenyi500.generate_dot('ErdosRenyi500.dot')

DOT file saved as ErdosRenyi50.dot
DOT file saved as ErdosRenyi200.dot
DOT file saved as ErdosRenyi500.dot


### Gilbert

In [64]:
# Grafo de Gilbert
graphGilbert50 = Graph(directed=False)
graphGilbert50.GilbertGraph(50, 0.3)
graphGilbert50.generate_dot('50Gilbert.dot')

graphGilbert200 = Graph(directed=False)
graphGilbert200.GilbertGraph(200, 0.05)
graphGilbert200.generate_dot('200Gilbert.dot')

graphGilbert500 = Graph(directed=False)
graphGilbert500.GilbertGraph(500, 0.1)
graphGilbert500.generate_dot('500Gilbert.dot')

DOT file saved as 50Gilbert.dot
DOT file saved as 200Gilbert.dot
DOT file saved as 500Gilbert.dot


### Barabasi-Albert

In [209]:
bara50 = Graph(directed=False)
bara50.BarabasiAlbertGraph(50,3)
bara50.generate_dot('bara50.dot')

bara200 = Graph(directed=False)
bara200.BarabasiAlbertGraph(200,maxGrade=4)
bara200.generate_dot('bara200.dot')

bara500 = Graph(directed=False)     
bara500.BarabasiAlbertGraph(500,6)
bara500.generate_dot('bara500.dot')

DOT file saved as bara50.dot
DOT file saved as bara200.dot
DOT file saved as bara500.dot


### Geográfico

In [81]:
geo50 = Graph(directed=False)
geo50.geographicGraph(n=50, r=0.2)
geo50.generate_dot('geo50.dot')

geo200 = Graph(directed=False)
geo200.geographicGraph(n=200, r=0.6)
geo200.generate_dot('geo200.dot')

geo500 = Graph(directed=False)
geo500.geographicGraph(n=500, r=0.1)
geo500.generate_dot('geo500.dot')

DOT file saved as geo50.dot
DOT file saved as geo200.dot
DOT file saved as geo500.dot


In [ ]:
# Grafo de Gilbert
graphGilbert = Graph(directed=False)
graphGilbert.GilbertGraph(200, 0.3)
graphGilbert.generate_dot('Gilbert.dot')


# Grafo de malla
graphMesh = Graph(directed=False)
graphMesh.meshGraph(25,20)
graphMesh.generate_dot('Mesh.dot')

# Grafo de ErdosRenyi
ErdosRenyi = Graph(directed=False)
ErdosRenyi.ErdosRenyiGraph(200, 60)
ErdosRenyi.generate_dot('ErdosRenyi.dot')


DOT file saved as Gilbert.dot
DOT file saved as Mesh.dot
DOT file saved as ErdosRenyi.dot
